<a href="https://colab.research.google.com/github/leeds1219/Article_Review/blob/main/AudioVisual_Grouping_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Audio-Visual Grouping Network for Sound Localization from Mixtures**
Shentong MO, Yapeng Tian


<img src = "https://drive.google.com/uc?id=1tPSL_zpnv99tELSvOgUkbmE69Xr6uvNF">


# Baseline Single-source Localization & Audio-Visual Encoder

$L_{baseline} = -\frac{1}{B}\Sigma_{b=1}^{B}log\frac{exp(\frac{1}{\tau}sim(F_{b}^{a},F_{b}^{v}))}{\Sigma_{m=1}^{B}exp(\frac{1}{τ}sim(F_{b}^{a},F_{m}^{v}))}$

B is batch, sim() denotes the max-pooled audio-visual cosine similarity of $F^{a}$ and $F^{v}=\{f_{v}^{p}\}_{p=1}^{P}$ across all P spatial locations, D is dimension size and $\tau$ is the temperature hyper-parameter

# Audio-Visual Class Tokens(Transformer Layers)

C is the source event categories

Aggregate audio-visual tokens $\{ \hat{c}_{i}^{a} \}_{i=1}^{C}$,
Global audio and spatial visual features $\hat{f}^{a}$


Align features and tokens from raw input of image and audio mixture with self attention transformers: $\hat{f}^{a}$, $\{ \hat{c}_{i}^{a} \}_{i=1}^{C}$ = $\{\phi(x_{j}^{a}, X^{a},X^{a})\}_{j=1}^{1+C}$

$\text{Attention}(Q, K, V) = \text{Softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$

$X^{a}$ = $\{x_{j}^{a}\}_{j=1}^{1+C}$ = [$f^{a}$;$\{c_{i}\}_{i=1}^{C}$] is the entire set of input features, $x_{j}^{a}$ is the indiviual element in the set.

${\phi^{a}}(x_{j}^{a}, X^{a},X^{a}) = \text{Softmax}\left(\frac{{x_{j}^{a}}{X^{a}}^T}{\sqrt{D}}\right) {X^{a}}
$

Individual source class probability $e_{i}$ = Softmax(FC(${c^{i}}$))

Cross entropy loss: $\Sigma_{i=1}^{C}CE(h_{i},e_{i})$

$h_{i}$ is the one hot encoding with its target category entry i as 1.

One-hot encoding is a technique used in machine learning and data processing to convert categorical data into a numerical format.

# Audio-Visual Grouping Module

Grouping blocks $g^{a}(\cdot)$, $g^{v}(\cdot)$

Similarity matrix

$A_{i}^{a}$ = Softmax($W_{q}^{a}\hat{f}^{a}\cdot W_{k}^{a}\hat{c}_{i}^{a}$)

$A_{p,i}^{v}$ = Softmax($W_{q}^{v}\hat{f}_{p}^{v}\cdot W_{k}^{v}\hat{c}_{i}^{v}$)

Weighted Sum calculation with the similarity matrix to gain Categori-Aware Representations

$g_{i}^{a}$ = $g^{a}(\hat{f}^{a},\hat{c}_{i}^{a})$ = $\hat{c}_{i}^{a}$ + $W_{o}^{a}$$A_{i}^{a}\frac{W_{v}^{a}\hat{f}^{a}}{A_{i}^{a}}$

$g_{i}^{v}$ = $g^{a}(\{\hat{f}_{p}^{v}\}_{p=1}^{P},\hat{c}_{i}^{v})$

= $\hat{c}_{i}^{v}$ + $W_{o}^{v}$$\frac{\Sigma_{p=1}^{P}A_{p,i}^{v}{W_{v}^{v}\hat{f}_{p}^{v}}}{{\Sigma_{p=1}^{P}A_{p,i}^{v}}}$

Binary probability is defined getting the grouping as input $p_{i}^{a}$ = Sigmoid(FC($g_{i}^{a}$))

Binary cross-entropy for multiple audio sources in a mixture, multi-label classification problem

$L_{group}$ = $\Sigma_{i=1}^{C}\{CE(h_{i},e_{i})+BCE(y_{i},p_{i}^{a})+BCE(y_{i},p_{i}^{v})\}$


# Localization

Audio-visual similarity is calculated by max pooling audio-visual cosine similarities of the class-aware audio features $g_{n}^{a}$ and the spatial-level

$L_{loc}$ = $-\frac{1}{BN}\Sigma_{b=1}^{B}\Sigma_{n=1}^{N}log\frac{exp(\frac{1}{τ}sim(F_{b,n}^{a},F_{b,n}^{v}))}{\Sigma_{m=1}^{B}exp(\frac{1}{τ}sim(F_{b,n}^{c},F_{m,n}^{v}))}$

Representations for N source embeddings are chosen from C categories according to the corresponding GT class

B is mini-batch size, N is sources in each batch

End-to-end loss $L$ = $L_{loc}+L_{group}$

# Inference

Use audio-visual cosine similarity map between class-aware audio-visual representations to genrate $n$the source localization map with $P$ locations.

The final localization map is gerated through bilinear interpolation of the similarity map.

Bilinear interpolation is particularly useful in RoI Align when estimating values at fractional coordinates within feature maps, providing a more accurate representation of the spatial information than methods like max pooling or average pooling.